In [1]:
import numpy as np
import pandas as pd


In [2]:
import csv
import numpy as np

cnt = 1
csv_file_path = 'crop_production.csv'
states = {}
districts = {}
season_to_number = {}
crops = {}
with open(csv_file_path, 'r') as file:
    csv_reader = csv.reader(file)
    for row in  csv_reader:
        cnt = cnt+1
        if cnt == 1 : continue
        states[row[0].strip()] = 1
        districts[row[1].strip()] = 1
        season_to_number[row[3].strip()] = 1
        crops[row[4].strip()] = 1
        
cnt = 1
for val in states.keys():
    states[val] = cnt
    cnt = cnt+1
cnt = 1
for val in districts.keys():
    districts[val] = cnt
    cnt = cnt+1
cnt = 1

for val in season_to_number.keys():
    season_to_number[val] = cnt
    cnt = cnt+1
cnt = 1
for val in crops.keys():
    crops[val] = cnt
    cnt = cnt+1


# New idea is to divide the crop produce by area so that we can predict for each district what yield would it give per area then
# multiply by area.

In [139]:
import csv
import numpy as np
from sklearn.model_selection import train_test_split


csv_file_path = 'crop_production.csv'

y = []
data = []
cnt = 0
with open(csv_file_path, 'r') as file:
    csv_reader = csv.reader(file)
    for row in csv_reader:
        cnt = cnt+1
        if cnt == 1 : continue
        try:
            val1 = float(row[6].strip())
            val2 = float(row[5])
            y.append(val1/val2)
        except Exception as e:
            # print(e)
            continue
        temp_row = row
        
        temp_row[1] = districts[temp_row[1].strip()]
        temp_row[3] = season_to_number[temp_row[3].strip()]
        temp_row[4] = crops[temp_row[4].strip()]
        
        temp1 = row[1:2]
        temp2 = row[3:5]
        
        data.append(temp1+temp2)
       
y = np.array(y)

data = np.array(data)
print(data.shape)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_normalized = scaler.fit_transform(data)
# i = 7
# print(data[i])
# print(y[i])


X_train, X_test, y_train, y_test = train_test_split(X_train_normalized, y, test_size = .25, random_state = 18)


(242361, 3)


In [4]:
# n_estimators=1000, max_depth=7, eta=0.1, subsample=0.7, colsample_bytree=0.8 

In [227]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

param_grid_gb = {
    "min_child_weight" : [46],
    "max_depth" : [8],
    "gamma" : [1e-01],
    "subsample" :[1],
    "colsample_bytree" : [1],
    "reg_alpha": [1e-04]
}

model = XGBRegressor()
mse_grid = GridSearchCV(estimator=model, param_grid=param_grid_gb, scoring='neg_mean_squared_error', cv=5, verbose=1, n_jobs=-1)

mse_grid.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=20)

print("Best parameters:", mse_grid.best_params_)


# Procedure: First tune min_child_weight and max_depth
# Tune gamma
# Tune subsample and colsample
# Tune regularization (alpha paramater)
# Tune the estimators as required

# Previous run (27m 35.4s) Best parameter:  {'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 200, 'subsample': 1}
# Best parameter:  {'colsample_bytree': 1e-05, 'gamma': 0, 'learning_rate': 1e-07, 'max_depth': 1, 'min_child_weight': 1, 'n_estimators': 100, 'reg_alpha': 1e-07, 'subsample': 1}


Fitting 5 folds for each of 3 candidates, totalling 15 fits
[0]	validation_0-rmse:973.14026
[1]	validation_0-rmse:940.09787
[2]	validation_0-rmse:921.49531
[3]	validation_0-rmse:912.53384
[4]	validation_0-rmse:911.49290
[5]	validation_0-rmse:912.14340
[6]	validation_0-rmse:909.75564
[7]	validation_0-rmse:911.11095
[8]	validation_0-rmse:909.23104
[9]	validation_0-rmse:909.43806
[10]	validation_0-rmse:910.35499
[11]	validation_0-rmse:910.00170
[12]	validation_0-rmse:909.05417
[13]	validation_0-rmse:908.84222
[14]	validation_0-rmse:910.38034
[15]	validation_0-rmse:910.14031
[16]	validation_0-rmse:910.44426
[17]	validation_0-rmse:910.38267
[18]	validation_0-rmse:911.39020
[19]	validation_0-rmse:911.55171
[20]	validation_0-rmse:909.75704
[21]	validation_0-rmse:910.88653
[22]	validation_0-rmse:910.86693
[23]	validation_0-rmse:911.56264
[24]	validation_0-rmse:912.19541
[25]	validation_0-rmse:912.12390
[26]	validation_0-rmse:912.11604
[27]	validation_0-rmse:912.09234
[28]	validation_0-rmse:912

In [6]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
# from sklearn.model_selection import RandomizedSearchCV

# param_random_gb = {
#               'learning_rate': np.arange(0.05,0.15, 0.05),
#               # 'n_estimators' : np.arange(200,1000,100)
#             #   'subsample' : np.arange(0.9,1, 0.05),
#             #   "colsample_bytree" : np.arange(0.9,1,0.05)
#               }

# # Regressor Instantiation
# gb = XGBRegressor()

# # Here we specified a total of 25 iterations
# mse_random = RandomizedSearchCV(estimator = gb, param_distributions= param_random_gb, n_iter = 25, 
#                                 scoring = 'neg_mean_squared_error', cv = 4, verbose = 1)

# mse_random.fit(X_train_normalized1,y_train1)

# print("Best parameter: ", mse_random.best_params_)

In [228]:
# Best parameter:  {'colsample_bytree': 1, 'gamma': 1e-05, 'learning_rate': 1e-07, 'max_depth': 4, 'min_child_weight': 12, 'n_estimators': 100, 'reg_alpha': 1e-08, 'subsample': 0.9}


model = XGBRegressor(n_estimators=1000,eta = 1e-03,min_child_weight = 46, max_depth = 8,gamma = 1e-01
                     ,colsample_bytree = 1 , subsample = 1 , reg_alpha = 1e-07)
# eval_set = [(X_test,y_test)]
model.fit   (X_train,y_train,eval_set=[(X_train,y_train),(X_test,y_test)],early_stopping_rounds=1000,)

# print("Best parameter: ", mse_grid.best_params_)

[0]	validation_0-rmse:732.35116	validation_1-rmse:1034.18954
[1]	validation_0-rmse:732.02924	validation_1-rmse:1033.95929
[2]	validation_0-rmse:731.70782	validation_1-rmse:1033.72948


c:\Python311\Lib\site-packages\xgboost\sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[3]	validation_0-rmse:731.38688	validation_1-rmse:1033.49998
[4]	validation_0-rmse:731.06644	validation_1-rmse:1033.27094
[5]	validation_0-rmse:730.74650	validation_1-rmse:1033.04226
[6]	validation_0-rmse:730.42704	validation_1-rmse:1032.81396
[7]	validation_0-rmse:730.10808	validation_1-rmse:1032.58608
[8]	validation_0-rmse:729.78960	validation_1-rmse:1032.35854
[9]	validation_0-rmse:729.47161	validation_1-rmse:1032.13143
[10]	validation_0-rmse:729.15412	validation_1-rmse:1031.90469
[11]	validation_0-rmse:728.83709	validation_1-rmse:1031.67837
[12]	validation_0-rmse:728.52057	validation_1-rmse:1031.45240
[13]	validation_0-rmse:728.20452	validation_1-rmse:1031.22680
[14]	validation_0-rmse:727.88897	validation_1-rmse:1031.00161
[15]	validation_0-rmse:727.57391	validation_1-rmse:1030.77679
[16]	validation_0-rmse:727.25932	validation_1-rmse:1030.55239
[17]	validation_0-rmse:726.94521	validation_1-rmse:1030.32831
[18]	validation_0-rmse:726.63160	validation_1-rmse:1030.10466
[19]	validation

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False, eta=0.001,
             eval_metric=None, feature_types=None, gamma=0.1, gpu_id=None,
             grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=8, max_leaves=None,
             min_child_weight=46, missing=nan, monotone_constraints=None,
             n_estimators=1000, n_jobs=None, num_parallel_tree=None,
             predictor=None, ...)

In [223]:
# training_error = 0
from sklearn.metrics import mean_squared_error
print("Training error:")
print(mean_squared_error(y_train,model.predict(X_train)))
print("Test error:")
print(mean_squared_error(y_test,model.predict(X_test)))
# Previously
# Training error:
# 2041076666.4211206
# Test error:
# 71631908865818.11
# Overfit
# Now
# Training error:
# 444956956552237.3
# Test error:
# 61499447426576.72
# Ok

Training error:
329396.57724061504
Test error:
854885.8564783292


In [230]:
# This will be used for user inputs
# State_Name = states['Andaman and Nicobar Islands']
District_Name = districts['NICOBARS']
# Crop_Year = 2000
Season = season_to_number['Kharif']
Crop = crops['Sugarcane']
Area = 102.00
# print(State_Name,District_Name,Crop_Year,Season,Crop,Area)
X_predict = np.zeros((1,3))
X_predict[0] = np.array([District_Name,Season,Crop])
X_pred_normalized = scaler.transform(X_predict)

Production = model.predict(X_pred_normalized)
print(X_pred_normalized)
print(Production*Area)

[[-1.65923103 -0.97610996 -1.21405365]]
[189.29796]


In [11]:
import pickle
filename = "finalized_model.sav"
# pickle.dump(model, open(filename, 'wb'))
 
# some time later...
 
# load the model from disk
model = pickle.load(open(filename, 'rb'))
# result = loaded_model.score(X_test, Y_test)

In [84]:
import pickle
filename = "finalized_model2.sav"
pickle.dump(model, open(filename, 'wb'))
 
# some time later...
#  Does not perform well on small aread like 1 10 100
# load the model from disk
# model = pickle.load(open(filename, 'rb'))
# result = loaded_model.score(X_test, Y_test)

In [163]:
import pickle
filename = "finalized_model3.sav"
pickle.dump(model, open(filename, 'wb'))
 
# some time later...
#  Does not perform well on small aread like 1 10 100
# load the model from disk
# model = pickle.load(open(filename, 'rb'))
# result = loaded_model.score(X_test, Y_test)

In [ ]:
# Overall modle
import pickle
filename = "finalized_model4.sav"
pickle.dump(model, open(filename, 'wb'))
 